# Setup

In [1]:
from piper.defaults import *

piper v0.1.0: Monday, 29 March 2021 19:05:11


# Test data 

In [2]:
%%piper
r812 <- pd.read_excel('inputs/R812.xlsx', sheet_name='query')
>> clean_columns() 
# >> count('sprint')
>> where("sprint == 'R812'")
# >> head()

# Setup

## Pivot table vs groupby

Pivot table vs groupby with multi-key and unstack and crosstab are functionally the same.<br>
Pivot allows automatic totals in both axes ... groupby does not.

### pivot_table

In [3]:
count(r812, 'functional_area')

,n,%,cum %
functional_area,,,
Logistics,50,56.82,56.82
ARO,14,15.91,72.73
Order Management,10,11.36,84.09
eCommerce,8,9.09,93.18
Finance,6,6.82,100.00


In [4]:
%%piper

r812.fillna(0)
>> pivot_table(index='functional_area',
                  columns='status', values='object',
                  fill_value=0,
                  dropna=False, aggfunc=np.size)
>> adorn()

Use %piper/%%piper --info to see rendered pandas pipe statement


name 'pivot_table' is not defined


### Groupby / unstack 

In [5]:
(r812.fillna('').groupby(['functional_area', 'status'])['status'].count()
 .unstack(level=1)
 .fillna(0)
 .astype(int))

status,,Baxter Code Review Complete,Development Complete,L3 In Progress,L3 Pass
functional_area,,,,,
ARO,0,2,0,8,4
Finance,0,5,1,0,0
Logistics,1,4,45,0,0
Order Management,0,0,10,0,0
eCommerce,0,0,8,0,0


### Crosstab

In [6]:
%%piper
pd.crosstab(r812.functional_area.fillna(''), r812.status.fillna(''))
>> adorn(axis='both')

,,Baxter Code Review Complete,Development Complete,L3 In Progress,L3 Pass,All
ARO,0,2,0,8,4,14
Finance,0,5,1,0,0,6
Logistics,1,4,45,0,0,50
Order Management,0,0,10,0,0,10
eCommerce,0,0,8,0,0,8
All,1,11,64,8,4,88


In [7]:
r812.query("~status.isin(['L3 In Progress', 'L3 Pass',\
           'Baxter Code Review Complete', 'Development Complete'])")

,object,gxp_critical_flag,object_type,fs_doc_nbr,ts_doc_nbr,sprint,testing_sprint,phase,description,status,l3_defect_number,l3_defect_status,deployed_to_ut,"critical_for_go_live_(l,_m,_h)",deployed_to_ppd,functional_area,main_function,baxter_functional_owner_name,l2_status,l2_testing_kpit_testing_comments,app_created_by,app_modified_by,baxter_tech_lead_name,code_review_comments,code_reviewer,compliance_asset_id,content_type,country,created,created_by,estimated_dev_end_date,estimated_dev_start_date,fsts_name/_number,id,interface_type,item_child_count,kpit_technical_lead,label_applied_by,label_setting,modified,modified_by,omw_project,purpose,ready_for_baxter_tech_review,ready_for_euut_promotion,retention_label,retention_label_applied,retrofit_comments,sprint_track,test_script_id_9_2,test_script_name,version_projects,global,global_needed_in_production,number,item_type,path
2888,P564915A,NaN,APPL,NaN,NaN,R812,NaN,NaN,Work with Shipments - Baxter EMEA,NaN,NaN,NaN,0.0,NaN,0.0,Logistics,NaN,Grant Wallis,L2 Not required,NaN,NaN,NaN,Terry Greenard / Vikas Verma,NaN,NaN,NaN,Item,NaN,2020-08-11 05:13:56,"Verma, Vikas",NaN,NaN,NaN,2953,NaN,0,NaN,NaN,NaN,2020-08-11 05:21:13,"Verma, Vikas",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,Item,sites/erp92upgrade/erp92emea/Lists/EMEA Retrofit Object Lifecycle Tracker


## L2 Status

In [8]:
(r812.fillna('').groupby(['functional_area', 'l2_status'])['l2_status'].count()
 .unstack(level=0)
 .fillna(0)
 .astype(int))

functional_area,ARO,Finance,Logistics,Order Management,eCommerce
l2_status,,,,,
L2 Not required,14,6,50,10,8


In [9]:
%%piper
count(r812, 'l2_status', totals=True)

,n,%,cum %
L2 Not required,88,100.0,100.0
Total,88,100.0,
